In [ ]:
!pip install pyAudioAnalysis
!pip install eyed3
!pip install pydub
!pip install opensmile

     |████████████████████████████████| 41.3 MB 1.4 MB/s 
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.14-py3-none-any.whl size=41264389 sha256=09a4c8777e395513cb4ce4fb0a80cd31186f9628bc0df7248b15c5137a02b131
  Stored in directory: /root/.cache/pip/wheels/85/b4/52/c6a5bdc8517a9bc4cc95aca9839f118d25139aee7881a8d210
Successfully built pyAudioAnalysis
     |████████████████████████████████| 245 kB 4.4 MB/s 
     |████████████████████████████████| 242 kB 48.1 MB/s 
  Attempting uninstall: coverage
    Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires coverage==3.7.1, but you have coverage 5.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incomp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
#Librerias
import eyed3
import pydub
from pyAudioAnalysis import ShortTermFeatures as aF
from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import MidTermFeatures as mF 
import IPython
import opensmile

import numpy as np
import plotly.graph_objects as go
import plotly
import sklearn
import pandas as pd
import seaborn as sns
import librosa
import librosa.display
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from statsmodels.formula.api import logit

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    
/usr/local/lib/python3.7/dist-packages/pyAudioAnalysis/../statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# **CARGA DE DATOS**

Cargamos la ruta donde se encuentra nuestras imagenes enticketadas por el nombre

In [ ]:
IEMOCAP = pd.read_csv("iemocap_full_dataset.csv")


In [ ]:
IEMOCAP

,session,method,gender,emotion,n_annotators,agreement,path
0,1,script,F,neu,3,3,Session1/sentences/wav/Ses01F_script02_1/Ses01...
1,1,script,F,fru,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...
2,1,script,F,xxx,0,0,Session1/sentences/wav/Ses01F_script02_1/Ses01...
3,1,script,F,sur,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...
4,1,script,F,neu,3,2,Session1/sentences/wav/Ses01F_script02_1/Ses01...
...,...,...,...,...,...,...,...
10034,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10035,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10036,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10037,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...


Filtramos para trabajar solo los audios que se obtuvieron mediante el metodo "IMPRO"

In [ ]:
IEMOCAP =IEMOCAP[IEMOCAP['method'] == 'impro']
IEMOCAP

,session,method,gender,emotion,n_annotators,agreement,path
71,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
72,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
73,1,impro,F,fru,4,2,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
74,1,impro,F,xxx,0,0,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
75,1,impro,F,xxx,0,0,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
...,...,...,...,...,...,...,...
10034,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10035,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10036,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10037,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...


Filtramos nuestro dataset con las emociones que vamos a trabajar

In [ ]:
#IEMOCAP=IEMOCAP[IEMOCAP['emotion'] in ('neu','hap','sad','ang')]
IEMOCAP = IEMOCAP.query('emotion in ("neu","hap","sad","ang")')
IEMOCAP

,session,method,gender,emotion,n_annotators,agreement,path
71,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
72,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
76,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
77,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
80,1,impro,F,neu,4,2,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
...,...,...,...,...,...,...,...
10034,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10035,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10036,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
10037,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...


Reseteamos los index

In [ ]:
IEMOCAP=IEMOCAP.reset_index()
IEMOCAP

,index,session,method,gender,emotion,n_annotators,agreement,path
0,71,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
1,72,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
2,76,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
3,77,1,impro,F,neu,4,3,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
4,80,1,impro,F,neu,4,2,Session1/sentences/wav/Ses01F_impro04/Ses01F_i...
...,...,...,...,...,...,...,...,...
2275,10034,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
2276,10035,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
2277,10036,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...
2278,10037,5,impro,F,neu,3,2,Session5/sentences/wav/Ses05F_impro06/Ses05F_i...


# **Funcion para extraer las caracteristicas del audio**

In [ ]:
import audiofile
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

In [ ]:
def ExtraerCaracteristicas ( df ): #ingresa df:Dataframe con los path de las rutas de audio , sale dataAudio con las caracteristicas LowLevelDescriptors
  dataAudio = pd.DataFrame(columns =['Loudness_sma3',
 'alphaRatio_sma3',
 'hammarbergIndex_sma3',
 'slope0-500_sma3',
 'slope500-1500_sma3',
 'spectralFlux_sma3',
 'mfcc1_sma3',
 'mfcc2_sma3',
 'mfcc3_sma3',
 'mfcc4_sma3',
 'F0semitoneFrom27.5Hz_sma3nz',
 'jitterLocal_sma3nz',
 'shimmerLocaldB_sma3nz',
 'HNRdBACF_sma3nz',
 'logRelF0-H1-H2_sma3nz',
 'logRelF0-H1-A3_sma3nz',
 'F1frequency_sma3nz',
 'F1bandwidth_sma3nz',
 'F1amplitudeLogRelF0_sma3nz',
 'F2frequency_sma3nz',
 'F2bandwidth_sma3nz',
 'F2amplitudeLogRelF0_sma3nz',
 'F3frequency_sma3nz',
 'F3bandwidth_sma3nz',
 'F3amplitudeLogRelF0_sma3nz'], index = range(df.shape[0]))


  #for n in range(ravdess_df.shape[0]):
  for n in range(df.shape[0]):
    try:
      signal, sampling_rate = audiofile.read("/content/drive/MyDrive/IEMOCAP_SinRuidoExtremo/"+df.path[n] ,  always_2d=True)
      fn= smile.process_signal(signal, sampling_rate)
      df_reset=fn.reset_index(drop=True)


      dataAudio.iloc[n] = df_reset.mean()


    except ValueError:
      print(n)
    
  return dataAudio


In [ ]:
dato_audio = ExtraerCaracteristicas(IEMOCAP)
dato_audio

,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,mfcc2_sma3,mfcc3_sma3,mfcc4_sma3,...,logRelF0-H1-A3_sma3nz,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2bandwidth_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3bandwidth_sma3nz,F3amplitudeLogRelF0_sma3nz
0,0.579245,-8.882022,24.044399,0.044476,-0.013207,0.274361,32.555378,-9.019608,1.697636,-20.344101,...,16.608719,542.402893,1276.165405,-63.632744,1444.010742,952.063232,-67.382561,2632.663818,751.134277,-78.581894
1,0.767835,-9.051714,22.091627,0.063348,-0.016865,0.327119,35.886063,-16.818283,8.180295,-15.283859,...,18.697691,444.075378,1326.335205,-0.416451,1433.251099,1024.477417,-8.889482,2423.309326,1046.052734,-19.2645
2,0.367772,-15.963787,22.671675,0.044672,-0.017057,0.158067,22.649122,12.147881,29.80163,-8.305227,...,25.502153,345.649353,1098.296631,-0.025989,1524.231567,697.42926,-24.384007,2356.639404,860.312134,-24.546804
3,0.946751,-7.900775,18.45911,0.062249,-0.017104,0.549107,26.662092,-19.264708,4.732305,-23.501585,...,15.2069,564.915405,1190.81311,-23.549522,1579.761597,870.1203,-30.367449,2635.749268,891.721863,-38.367031
4,0.579799,-11.408775,22.375961,0.059403,-0.01189,0.241613,39.11797,-17.417124,14.777751,-9.958256,...,12.050492,444.503693,1326.991455,-55.209782,1541.463013,1315.637939,-65.579811,2233.791504,1487.267334,-67.110168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,0.112906,-20.871483,30.994869,-0.019765,-0.02513,0.026738,37.138538,4.838092,24.17952,-5.39659,...,27.132988,478.253815,1346.999023,-15.201614,1517.193726,1572.817627,-28.424891,2337.456055,1207.390259,-33.819347
2276,0.128075,-12.919745,22.312456,0.002691,-0.010539,0.032478,29.998785,-2.978961,19.097158,-6.8337,...,9.772537,451.888214,1263.029541,-122.362221,1415.824829,1011.512268,-127.29776,2407.312256,887.206299,-130.132736
2277,0.106923,-12.114124,19.384136,0.000115,-0.009738,0.031964,26.198347,0.293229,22.601936,-7.257889,...,6.950793,486.365601,1254.163574,-139.897659,1481.072266,974.3573,-143.875885,2428.849365,837.190613,-144.56282
2278,0.077421,-13.84972,20.871487,-0.008137,-0.011364,0.020847,27.460932,-0.524265,20.104769,-9.971001,...,3.746234,517.403442,1307.88623,-172.239151,1474.410889,1088.902832,-173.91037,2437.568848,776.776184,-174.312531


validamos si no hay vacios

In [ ]:
print(dato_audio.isnull().sum())

Loudness_sma3                  0
alphaRatio_sma3                0
hammarbergIndex_sma3           0
slope0-500_sma3                0
slope500-1500_sma3             0
spectralFlux_sma3              0
mfcc1_sma3                     0
mfcc2_sma3                     0
mfcc3_sma3                     0
mfcc4_sma3                     0
F0semitoneFrom27.5Hz_sma3nz    0
jitterLocal_sma3nz             0
shimmerLocaldB_sma3nz          0
HNRdBACF_sma3nz                0
logRelF0-H1-H2_sma3nz          0
logRelF0-H1-A3_sma3nz          0
F1frequency_sma3nz             0
F1bandwidth_sma3nz             0
F1amplitudeLogRelF0_sma3nz     0
F2frequency_sma3nz             0
F2bandwidth_sma3nz             0
F2amplitudeLogRelF0_sma3nz     0
F3frequency_sma3nz             0
F3bandwidth_sma3nz             0
F3amplitudeLogRelF0_sma3nz     0
dtype: int64


# Agregramos la columna de emociones

In [ ]:
dato_audio["Emociones"] = IEMOCAP.emotion


In [ ]:
dato_audio

,Loudness_sma3,alphaRatio_sma3,hammarbergIndex_sma3,slope0-500_sma3,slope500-1500_sma3,spectralFlux_sma3,mfcc1_sma3,mfcc2_sma3,mfcc3_sma3,mfcc4_sma3,...,F1frequency_sma3nz,F1bandwidth_sma3nz,F1amplitudeLogRelF0_sma3nz,F2frequency_sma3nz,F2bandwidth_sma3nz,F2amplitudeLogRelF0_sma3nz,F3frequency_sma3nz,F3bandwidth_sma3nz,F3amplitudeLogRelF0_sma3nz,Emociones
0,0.579245,-8.882022,24.044399,0.044476,-0.013207,0.274361,32.555378,-9.019608,1.697636,-20.344101,...,542.402893,1276.165405,-63.632744,1444.010742,952.063232,-67.382561,2632.663818,751.134277,-78.581894,neu
1,0.767835,-9.051714,22.091627,0.063348,-0.016865,0.327119,35.886063,-16.818283,8.180295,-15.283859,...,444.075378,1326.335205,-0.416451,1433.251099,1024.477417,-8.889482,2423.309326,1046.052734,-19.2645,neu
2,0.367772,-15.963787,22.671675,0.044672,-0.017057,0.158067,22.649122,12.147881,29.80163,-8.305227,...,345.649353,1098.296631,-0.025989,1524.231567,697.42926,-24.384007,2356.639404,860.312134,-24.546804,neu
3,0.946751,-7.900775,18.45911,0.062249,-0.017104,0.549107,26.662092,-19.264708,4.732305,-23.501585,...,564.915405,1190.81311,-23.549522,1579.761597,870.1203,-30.367449,2635.749268,891.721863,-38.367031,neu
4,0.579799,-11.408775,22.375961,0.059403,-0.01189,0.241613,39.11797,-17.417124,14.777751,-9.958256,...,444.503693,1326.991455,-55.209782,1541.463013,1315.637939,-65.579811,2233.791504,1487.267334,-67.110168,neu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,0.112906,-20.871483,30.994869,-0.019765,-0.02513,0.026738,37.138538,4.838092,24.17952,-5.39659,...,478.253815,1346.999023,-15.201614,1517.193726,1572.817627,-28.424891,2337.456055,1207.390259,-33.819347,neu
2276,0.128075,-12.919745,22.312456,0.002691,-0.010539,0.032478,29.998785,-2.978961,19.097158,-6.8337,...,451.888214,1263.029541,-122.362221,1415.824829,1011.512268,-127.29776,2407.312256,887.206299,-130.132736,neu
2277,0.106923,-12.114124,19.384136,0.000115,-0.009738,0.031964,26.198347,0.293229,22.601936,-7.257889,...,486.365601,1254.163574,-139.897659,1481.072266,974.3573,-143.875885,2428.849365,837.190613,-144.56282,neu
2278,0.077421,-13.84972,20.871487,-0.008137,-0.011364,0.020847,27.460932,-0.524265,20.104769,-9.971001,...,517.403442,1307.88623,-172.239151,1474.410889,1088.902832,-173.91037,2437.568848,776.776184,-174.312531,neu


# Exportamos la data en formato EXCEL

In [ ]:
dato_audio.to_excel('IEMOCAP-SINRUIDO-dato_audioopensmile.xlsx')

# =============================FIN========================================